In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geojsonio
import config

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [25]:
ge = pd.read_csv("data/election-data-2004-2018.csv")

In [139]:
ge.head(3).T

,0,1,2
year,2004,2004,2004
state,Perlis,Perlis,Perlis
seat_code_no,1,1,2
seat_code,P1,P1,P2
seat_name,Padang Besar,Padang Besar,Kangar
candidate,Datuk Seri Azmi Khalid,Wan Kharizal Wan Khazim,Datuk Radzi Sheikh Ahmad
party,UMNO,PAS,UMNO
coalition,BN,Pakatan,BN
winlose,1,0,1
is_uncontested,0,0,0


1. Get only results from GE14
2. Get only winners
3. Create winner JSONs for each party

In [63]:
ge11 = ge[ge["year"] == 2004]
ge12 = ge[ge["year"] == 2008]
ge13 = ge[ge["year"] == 2013]
ge14 = ge[ge["year"] == 2018]

In [80]:
for year in ge["year"].unique():
    ge_year = ge[ge["year"] == year]
    parties = ge_year["party"].unique()
    print("Competing parties in year {}:\n{}\n".format(year, ", ".join(parties)))

Competing parties in year 2004:
UMNO, PAS, MCA, PKR, Ind, DAP, GERAKAN, PPP, MIC, MDP, IND, PBS, SETIA, UPKO, SAPP, PBRS, LDP, PASOK, SPDP, SNAP, PBB, SUPP, STAR, PBDS

Competing parties in year 2008:
UMNO, PAS, PKR, MCA, IND, DAP, GERAKAN, PPP, MIC, PRM, PBS, BERSEKUTU, UPKO, SAPP, PASOK, PBRS, LDP, SPDP, SNAP, PBB, SUPP, PRS

Competing parties in year 2013:
PAS, UMNO, IND, PKR, MCA, BERJASA, BERSAMA, KITA, DAP, PCM, MIC, GERAKAN, PPP, SAPP, STAR, PBS, UPKO, PBRS, LDP, SPDP, PBB, SUPP, PRS, SWP

Competing parties in year 2018:
UMNO, BERSATU, PAS, PKR, AMANAH, MCA, BEBAS, PRM, DAP, PCM, MU, GERAKAN, PFP, MIC, PSM, BERJASA, IKATAN, MYPPP, PAP, WARISAN, HR, PPRS, PBS, PCS, UPKO, SAPP, ANAKNEGERI, STAR, PBRS, LDP, PDP, PBB, SUPP, PRS, PBDSB, PBK, PEACE



In [128]:
ge14_seats_won = ge14[ge14["winlose"] == 1]

In [130]:
ge14_seats_bn = ge14_seats_won[ge14_seats_won["coalition"] == "BN"]
ge14_seats_ph = ge14_seats_won[ge14_seats_won["coalition"] == "Pakatan"]
ge14_seats_gs = ge14_seats_won[ge14_seats_won["coalition"] == "PAS"]

In [160]:
ge14_seats_bn["seat_code_no"][:3]

1503    1
1509    3
1521    7
Name: seat_code_no, dtype: int64

In [146]:
ge14_geo = gpd.read_file("data/parliamentary-boundaries-2018-working.geojson")

In [148]:
ge14_geo.head()

,KodPAR,Parliament,State,geometry
0,054,GERIK,PERAK,(POLYGON ((100.9933944673964 5.507324842759963...
1,055,LENGGONG,PERAK,(POLYGON ((100.9293339657613 5.514131158215433...
2,056,LARUT,PERAK,(POLYGON ((100.8672078753412 5.428777928590932...
3,058,BAGAN SERAI,PERAK,(POLYGON ((100.6241018705403 5.141014699246791...
4,057,PARIT BUNTAR,PERAK,(POLYGON ((100.4127561987131 4.992960828699954...


In [155]:
ge14_geo["KodPAR"] = pd.to_numeric(ge14_geo["KodPAR"])

In [168]:
ge14_geo_bn = ge14_geo[ge14_geo["KodPAR"].isin(ge14_seats_bn["seat_code_no"])]
ge14_geo_ph = ge14_geo[ge14_geo["KodPAR"].isin(ge14_seats_ph["seat_code_no"])]
ge14_geo_gs = ge14_geo[ge14_geo["KodPAR"].isin(ge14_seats_gs["seat_code_no"])]

In [172]:
ge14_geo_bn.head()

,KodPAR,Parliament,State,geometry
0,54,GERIK,PERAK,(POLYGON ((100.9933944673964 5.507324842759963...
1,55,LENGGONG,PERAK,(POLYGON ((100.9293339657613 5.514131158215433...
2,56,LARUT,PERAK,(POLYGON ((100.8672078753412 5.428777928590932...
3,58,BAGAN SERAI,PERAK,(POLYGON ((100.6241018705403 5.141014699246791...
6,61,PADANG RENGAS,PERAK,"(POLYGON ((100.8067462782922 4.85724527156128,..."


In [175]:
with open ("data/ge14-geo-bn.geojson", "w") as f:
    f.write(ge14_geo_bn.to_json())

In [ ]:
with open ("data/ge14-geo-ph.geojson", "w") as f:
    f.write(ge14_geo_ph.to_json())

In [ ]:
with open ("data/ge14-geo-gs.geojson", "w") as f:
    f.write(ge14_geo_gs.to_json())

In [2]:
plotly.tools.set_credentials_file(
    username=config.plotly_username, 
    api_key=config.plotly_api_key)
mapbox_access_token = config.mapbox_access_token

In [ ]:
# seats = gpd.read_file("data/parliamentary-boundaries-2018-working.geojson")

In [ ]:
# seats.head()

In [17]:
west_malaysia = dict(
    lat=4.0891679,
    lon=100.5595715
)

In [22]:
all_data_url = "https://raw.githubusercontent.com/theianchan/ge14-map/master/data/parliamentary-boundaries-2018-working.geojson"

In [24]:
data = go.Data([
    go.Scattermapbox(
        lat=west_malaysia["lat"],
        lon=west_malaysia["lon"],
        mode='markers',
    )
])
layout = go.Layout(
    height=600,
    autosize=True,
    hovermode="closest",
    mapbox=dict(
        layers=[
            dict(
                sourcetype="geojson",
                source=all_data_url,
                type="fill",
                color="rgba(40,0,113,0.8)"
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=west_malaysia["lat"],
            lon=west_malaysia["lon"],
        ),
        pitch=0,
        zoom=5.2,
        style="light"
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename="Prototype")

KeyError: 'data'